In [3]:
import numpy as np
import os,sys
import matplotlib.pyplot as plt

In [5]:

for idx in range(1,7):
    filedir = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass'
    filename = 'LiCl_H2O_25_0'+str(idx)+'.xyz'

    with open(os.path.join(filedir,filename),'r') as f:
        lines = f.readlines()[2:]
    lines = np.array([e.split() for e in lines])
    Li_idx = np.where(lines[:,0]=='Li')[0]
    Cl_idx = np.where(lines[:,0]=='Cl')[0]
    H_idx = np.where(lines[:,0]=='H')[0]
    O_idx = np.where(lines[:,0]=='O')[0]
    LiCl_num =len(Li_idx)
    H2O_num = len(O_idx)
    filehead = "LiCl\n  1.00000000000000\n"+\
    "\t14.5000000000020002    0.0000000000000000    0.0000000000000000\n"+\
    "\t0.0000000000000000   14.6999999999999993    0.0000000000000000\n"+\
    "\t0.0000000000000000    0.0000000000000000   14.5000000000000302\n"+\
    "   Li    Cl    O    H\n"+\
    "   "+str(LiCl_num)+"    "+str(LiCl_num)+"   "+str(H2O_num)+"   "+str(H2O_num*2)+"\n"+\
    "Cartesian\n"
    newlines = np.concatenate((lines[Li_idx],lines[Cl_idx],lines[O_idx],lines[H_idx]))
    newlines = [filehead]+['\t'.join(e[1:])+'\n' for e in newlines]
    filename_save = 'POSCAR'+str(idx)


    with open(os.path.join(filedir,filename_save),'w+') as f:
        f.writelines(newlines)

In [6]:
charge = {'Li':1,'Cl':-1,'O':-0.8476, 'H':0.4238}
mass = {'Li':6.941,'Cl':35.45,'O':15.999, 'H':1.00784}
for idx in range(1,7):
    filedir = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass'
    filename = 'LiCl_H2O_25_0'+str(idx)+'.xyz'

    with open(os.path.join(filedir,filename),'r') as f:
        lines = f.readlines()[2:]
    lines = np.array([e.split() for e in lines])
    Li_idx = np.where(lines[:,0]=='Li')[0]
    Cl_idx = np.where(lines[:,0]=='Cl')[0]
    H_idx = np.where(lines[:,0]=='H')[0]
    O_idx = np.where(lines[:,0]=='O')[0]
    
    Atoms = []
    Bonds = []
    Angles = []
    
    a_idx = 1
    b_idx = 1
    ang_idx = 1
    mol_idx = 1
    
    Atoms_lines = ['Atoms\n','\n']
    Bonds_lines = ['Bonds\n','\n']
    Angles_lines = ['Angles\n','\n']
    for li in Li_idx:
        #1       1       1       2       0.0     0.0     0.0
        Atoms_lines.append('\t'.join([str(a_idx),str(mol_idx),'1',str(charge['Li']),lines[li][1],lines[li][2],lines[li][3]])+'\n')
        a_idx+=1
    
    for cl in Cl_idx:
        #1       1       1       2       0.0     0.0     0.0
        Atoms_lines.append('\t'.join([str(a_idx),str(mol_idx),'2',str(charge['Cl']),lines[cl][1],lines[cl][2],lines[cl][3]])+'\n')
        a_idx+=1
    
    mol_idx += 1
    for i in range(len(O_idx)):
        O = lines[O_idx[i]]
        H1 = lines[H_idx[2*i]]
        H2 = lines[H_idx[2*i+1]]
        Atoms_lines.append('\t'.join([str(a_idx),str(mol_idx),'3',str(charge['O']),O[1],O[2],O[3]])+'\n')
        Atoms_lines.append('\t'.join([str(a_idx+1),str(mol_idx),'4',str(charge['H']),H1[1],H1[2],H1[3]])+'\n')
        Atoms_lines.append('\t'.join([str(a_idx+2),str(mol_idx),'4',str(charge['H']),H2[1],H2[2],H2[3]])+'\n')

        Bonds_lines.append('\t'.join([str(b_idx),'1',str(a_idx),str(a_idx+1)])+'\n')
        Bonds_lines.append('\t'.join([str(b_idx+1),'1',str(a_idx),str(a_idx+2)])+'\n')
        Angles_lines.append('\t'.join([str(ang_idx),'1',str(a_idx+1),str(a_idx),str(a_idx+2)])+'\n')
        
        ang_idx+=1
        mol_idx+=1
        b_idx+=2
        a_idx+=3
    
    #write head
    out_lines = ['\n','\n',str(len(lines))+' atoms\n',str(len(H_idx))+' bonds\n', str(len(O_idx))+' angles\n', '0 dihedrals\n']
    out_lines+= ['\n','4 atom types\n', '1 bond types\n','1 angle types\n','0 dihedral types\n','\n']
    out_lines+= ['0.0     14.5000000000020002  xlo xhi\n',
                 '0.0     14.6999999999999993  ylo yhi\n',
                 '0.0     14.5000000000000302  zlo zhi\n','\n']
    
    #write Atoms
    out_lines = out_lines + Atoms_lines + ['\n']+Bonds_lines+ ['\n']+Angles_lines
    
    
    with open(os.path.join(filedir,'data.'+str(idx)),'w+') as f:
        f.writelines(out_lines)
    
    

In [41]:

for idx in range(1,7):
    data_path = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass/RT/'+str(idx)+'case/lammps'
    filename = 'equ.xyz'

    with open(os.path.join(data_path,filename),'r') as f:
        lines = f.readlines()[-302:]
    lines = np.array([e.split() for e in lines])
    Li_idx = np.where(lines[:,0]=='Li')[0]
    Cl_idx = np.where(lines[:,0]=='Cl')[0]
    H_idx = np.where(lines[:,0]=='H')[0]
    O_idx = np.where(lines[:,0]=='O')[0]
    LiCl_num =len(Li_idx)
    H2O_num = len(O_idx)
    filehead = "LiCl\n  1.00000000000000\n"+\
    "\t14.5000000000020002    0.0000000000000000    0.0000000000000000\n"+\
    "\t0.0000000000000000   14.6999999999999993    0.0000000000000000\n"+\
    "\t0.0000000000000000    0.0000000000000000   14.5000000000000302\n"+\
    "   Li    Cl    O    H\n"+\
    "   "+str(LiCl_num)+"    "+str(LiCl_num)+"   "+str(H2O_num)+"   "+str(H2O_num*2)+"\n"+\
    "Cartesian\n"
    newlines = np.concatenate((lines[Li_idx],lines[Cl_idx],lines[O_idx],lines[H_idx]))
    newlines = [filehead]+['\t'.join(e[1:])+'\n' for e in newlines]
    filename_save = 'POSCAR'+str(idx)+'_afterMD'

    filedir = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass/RT/'
    with open(os.path.join(filedir,filename_save),'w+') as f:
        f.writelines(newlines)

In [30]:
data_path = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass/RT/2case/lammps'
filename = 'equ.xyz'
with open(os.path.join(data_path,filename),'r') as f:
    lines = f.readlines()[-302:]
lines

['Li 7.88132 10.4477 9.88594\n',
 'Li 3.03255 11.8109 3.99174\n',
 'Li 10.575 6.84061 10.8725\n',
 'Li 13.406 12.8408 13.1977\n',
 'Li 4.66148 3.89431 12.4347\n',
 'Li 11.6672 2.83949 5.51612\n',
 'Li 3.48553 11.5725 7.36682\n',
 'Li 8.35519 6.26801 7.05064\n',
 'Li 2.75005 5.8431 7.90715\n',
 'Li 9.18712 2.91996 13.9584\n',
 'Li 5.67975 7.92782 1.30581\n',
 'Li 13.2467 10.3271 6.36342\n',
 'Li 4.14826 10.8593 11.5714\n',
 'Cl 5.49653 10.9819 9.0643\n',
 'Cl 3.07319 12.6134 1.41037\n',
 'Cl 12.8268 6.37278 9.30228\n',
 'Cl 10.9157 12.0837 12.9049\n',
 'Cl 3.99311 3.27581 0.687265\n',
 'Cl 7.03076 1.53156 8.36107\n',
 'Cl 3.868 13.4723 5.80984\n',
 'Cl 8.90139 8.06694 9.26291\n',
 'Cl 0.701737 10.5468 4.58416\n',
 'Cl 9.09273 4.38112 11.6628\n',
 'Cl 5.00167 7.12842 3.75706\n',
 'Cl 13.5655 0.708642 12.8306\n',
 'Cl 7.9012 7.74392 14.4873\n',
 'O 4.36477 7.58205 7.83245\n',
 'H 5.24168 7.55958 7.35231\n',
 'H 4.52324 7.72521 8.80938\n',
 'O 7.11379 13.8571 13.7982\n',
 'H 6.76086 13.914